#### things to make modular
* query expansion
* query expansion result aggregation method
* using clean corpus
* retrieval method (dres/bm25)

#### imports

In [1]:
from coir.data_loader import get_tasks
from coir.evaluation import COIR
from coir.models import YourCustomDEModel
from datasets import load_dataset, Dataset, concatenate_datasets
import pandas as pd
from query_expansion import expand_queries_and_qrels
from functions import * 
from collections import defaultdict
import copy

In [2]:
cosqa_expl_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/Cosqa/CoSQA_explanations_query_code.csv'
cosqa_deepseek_updated_path='/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CS696DS-Oracle-Retrieving-Code-Explanations/Explanation_Generation/Cosqa/postprocessing/output/COSQA_deepseek_explanations_clean.csv'
cosqa_granite_updated_path='/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CS696DS-Oracle-Retrieving-Code-Explanations/Explanation_Generation/Cosqa/postprocessing/output/COSQA_granite_explanations_clean.csv'
cosqa_deepseek_temp_0_qe_path = '/work/pi_wenlongzhao_umass_edu/27/janet/query_expansion/results/cosqa/cosqa_queries_expanded_deepseek_temp_0.csv'
cosqa_queries_expanded_granite_temp_0pt5_mmr_2_path='/work/pi_wenlongzhao_umass_edu/27/janet/query_expansion/results/cosqa/cosqa_queries_expanded_granite_temp_0.5_mmr_2.csv'
csn_queries_expanded_granite_temp_0pt5_mmr_path = '/work/pi_wenlongzhao_umass_edu/27/janet/query_expansion/results/csn/csn_test_queries_expanded_granite_temp_0.5_mmr.csv'
csn_deepseek_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CS696DS-Oracle-Retrieving-Code-Explanations/Explanation_Generation/output/CSN_deepseek.csv'
csn_granite_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CS696DS-Oracle-Retrieving-Code-Explanations/Explanation_Generation/output/CSN_granite.csv'
csn_code_based_queries_expanded = '/work/pi_wenlongzhao_umass_edu/27/janet/query_expansion/results/csn/csn_test_queries_expanded_granite_code_based.csv'

#### No QE

In [3]:
dataset_name = "cosqa" #CodeSearchNet-python  or cosqa
tasks = load_data(dataset_name)
corpus, queries, qrels = tasks[dataset_name]

in tasks 
data loader init


0it [00:00, ?it/s]

In [8]:
df= pd.read_csv(cosqa_granite_updated_path) # csn_granite_path or cosqa_deepseek_updated_path
corpus = update_corpus_with_cleaned_code(corpus, df) # updating the corpus with no comments in code
tasks[dataset_name] = (corpus, queries, qrels)

#### Query Expansion

In [3]:
# # CSN code based qe
# csn_code_based_qe_df = pd.read_csv(csn_code_based_queries_expanded)
# csn_code_based_qe_df=csn_code_based_qe_df.rename(columns={'explanation': 'expanded_query'})
# csn_code_based_qe_df=csn_code_based_qe_df.rename(columns={'id': 'v_id'})
# csn_code_based_qe_df=csn_code_based_qe_df.rename(columns={'query_id': 'id'})

# dataset_name = "CodeSearchNet-python"
# tasks = load_data(dataset_name)
# corpus, queries, qrels = tasks[dataset_name]

# df= pd.read_csv(csn_granite_path) # csn_granite_path or cosqa_deepseek_updated_path
# corpus = update_corpus_with_cleaned_code(corpus, df) # updating the corpus with no comments in code
# tasks[dataset_name] = (corpus, queries, qrels)

# # updating queries for qe
# new_queries, new_qrels = expand_queries_and_qrels(csn_code_based_qe_df, qrels)
# tasks[dataset_name] = (corpus, new_queries, qrels)

# CSN
# csn_qe_df = pd.read_csv(csn_queries_expanded_granite_temp_0pt5_mmr_path)
# csn_qe_df=csn_qe_df.rename(columns={'mmr_explanation': 'expanded_query'})

# dataset_name = "CodeSearchNet-python"
# tasks = load_data(dataset_name)
# corpus, queries, qrels = tasks[dataset_name]

# df= pd.read_csv(csn_granite_path) # csn_granite_path or cosqa_deepseek_updated_path
# corpus = update_corpus_with_cleaned_code(corpus, df) # updating the corpus with no comments in code
# tasks[dataset_name] = (corpus, queries, qrels)

# # updating queries for qe
# new_queries, new_qrels = expand_queries_and_qrels(csn_qe_df, qrels)
# tasks[dataset_name] = (corpus, new_queries, qrels)

# COSQA
dataset_name = "cosqa"
tasks = load_data(dataset_name)
corpus, queries, qrels = tasks[dataset_name]

# expanded df
cosqa_qe_df = pd.read_csv(cosqa_queries_expanded_granite_temp_0pt5_mmr_2_path)
cosqa_qe_df = cosqa_qe_df.rename(columns={'mmr_explanation': 'expanded_query'})
cosqa_qe_df = cosqa_qe_df[cosqa_qe_df["id"].isin(queries.keys())]


# updating the corpus with no comments in code
df= pd.read_csv(cosqa_granite_updated_path)
corpus = update_corpus_with_cleaned_code(corpus, df) 

# updating queries for qe
new_queries, new_qrels = expand_queries_and_qrels(cosqa_qe_df, qrels)
tasks[dataset_name] = (corpus, new_queries, qrels)

in tasks 
data loader init


0it [00:00, ?it/s]

#### Explanations

In [4]:
# if you want to add explanations (deepseek / granite) (cosqa / csn)
tasks_expl = copy.deepcopy(tasks)
#add_expl(tasks, "CodeSearchNet-python", csn_deepseek_path, "explanation_deepseek_1") # adding explanation instead of code
add_expl(tasks_expl, "cosqa", cosqa_granite_updated_path, "explanation_granite_1_cleaned") # adding explanation instead of code

Total docs in corpus after replacement: 4419
Number of docs still missing 'text': 0


#### Running the pipeline

In [5]:
c,b,a = tasks[dataset_name]
f,e,d = tasks_expl[dataset_name]

In [6]:
c['d9318']

{'text': '\ndef _convert_to_array(array_like, dtype):\n\n    if isinstance(array_like, bytes):\n        return np.frombuffer(array_like, dtype=dtype)\n    return np.asarray(array_like, dtype=dtype)',
 'title': ''}

In [7]:
f['d9318']

{'text': 'This Python function, `_convert_to_array`, converts an input `array_like` object into a NumPy array. It checks if the input is a `bytes` object, and if so, uses `np.frombuffer` to convert it to a NumPy array. Otherwise, it uses `np.asarray` for conversion. The `dtype` parameter specifies the data type of the output array.',
 'title': ''}

In [8]:
from concurrent.futures import ThreadPoolExecutor, as_completed

run_params = [
    run("intfloat/e5-base-v2", tasks, "baseline", "dres", "cosqa/qe/granite_temp_0.5_mmr_2/median"),
    run("BAAI/bge-base-en", tasks, "baseline", "dres", "cosqa/qe/granite_temp_0.5_mmr_2/median"),
    run("intfloat/e5-base-v2", tasks_expl, "granite1", "dres", "cosqa/qe/granite_temp_0.5_mmr_2/median"),
    run("BAAI/bge-base-en", tasks_expl, "granite1", "dres", "cosqa/qe/granite_temp_0.5_mmr_2/median")
    
]

def run_wrapper(params):
    model_name, task, exp_type, retrieval, data_path = params
    return run(model_name, task, exp_type, retrieval, data_path)

# Run all in parallel
if __name__ == "__main__":
    max_workers = 4  # Adjust based on your machine's capacity

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(run_wrapper, params) for params in run_params]

        for future in as_completed(futures):
            try:
                result = future.result()
                print("Run completed successfully.")
            except Exception as e:
                print(f"Run failed: {e}")


YourCustomDEModel init
COIR init!
in evaluation.py: loading up dres

in evaluation.py: retrieving

in beir/retrieval/evaluation.py: loading up search

in exact_search.py
len of queries:  2500


Encoding batches:   0%|          | 0/10 [00:00<?, ?batch/s]

len of corpus:  4419
Encoding Corpus in batches... Warning: This might take a while!
Scoring Function: Cosine Similarity (cos_sim)
Encoding Batch 1/1...


Encoding batches:   0%|          | 0/18 [00:00<?, ?batch/s]

exact_search finished
in evaluation.py: evaluating

in beir/retrieval/evaluation.py: evaluating now

in combine scores by median
Retrieval evaluation results saved to results/cosqa/qe/granite_temp_0.5_mmr_2/median/baseline/dres/intfloat/e5-base-v2
{'cosqa': {'NDCG': {'NDCG@1': 0.32, 'NDCG@3': 0.44881, 'NDCG@5': 0.48703, 'NDCG@10': 0.5221, 'NDCG@100': 0.56746, 'NDCG@1000': 0.57463}, 'MAP': {'MAP@1': 0.32, 'MAP@3': 0.41667, 'MAP@5': 0.43807, 'MAP@10': 0.453, 'MAP@100': 0.46312, 'MAP@1000': 0.46339}, 'Recall': {'Recall@1': 0.32, 'Recall@3': 0.542, 'Recall@5': 0.634, 'Recall@10': 0.74, 'Recall@100': 0.944, 'Recall@1000': 1.0}, 'Precision': {'P@1': 0.32, 'P@3': 0.18067, 'P@5': 0.1268, 'P@10': 0.074, 'P@100': 0.00944, 'P@1000': 0.001}}}
YourCustomDEModel init
COIR init!
in evaluation.py: loading up dres

in evaluation.py: retrieving

in beir/retrieval/evaluation.py: loading up search

in exact_search.py
len of queries:  2500


...

exact_search finished
in evaluation.py: evaluating

in beir/retrieval/evaluation.py: evaluating now

in combine scores by median
Retrieval evaluation results saved to results/cosqa/qe/granite_temp_0.5_mmr_2/median/granite1/dres/intfloat/e5-base-v2
{'cosqa': {'NDCG': {'NDCG@1': 0.316, 'NDCG@3': 0.42224, 'NDCG@5': 0.46389, 'NDCG@10': 0.50478, 'NDCG@100': 0.55743, 'NDCG@1000': 0.56206}, 'MAP': {'MAP@1': 0.316, 'MAP@3': 0.39667, 'MAP@5': 0.41957, 'MAP@10': 0.43653, 'MAP@100': 0.44811, 'MAP@1000': 0.44829}, 'Recall': {'Recall@1': 0.316, 'Recall@3': 0.496, 'Recall@5': 0.598, 'Recall@10': 0.724, 'Recall@100': 0.962, 'Recall@1000': 0.998}, 'Precision': {'P@1': 0.316, 'P@3': 0.16533, 'P@5': 0.1196, 'P@10': 0.0724, 'P@100': 0.00962, 'P@1000': 0.001}}}
YourCustomDEModel init
COIR init!
in evaluation.py: loading up dres

in evaluation.py: retrieving

in beir/retrieval/evaluation.py: loading up search

in exact_search.py
len of queries:  2500


Encoding batches:   0%|          | 0/10 [00:00<?, ?batch/s]

len of corpus:  4419
Encoding Corpus in batches... Warning: This might take a while!
Scoring Function: Cosine Similarity (cos_sim)
Encoding Batch 1/1...


Encoding batches:   0%|          | 0/18 [00:00<?, ?batch/s]

exact_search finished
in evaluation.py: evaluating

in beir/retrieval/evaluation.py: evaluating now

in combine scores by median
Retrieval evaluation results saved to results/cosqa/qe/granite_temp_0.5_mmr_2/median/granite1/dres/BAAI/bge-base-en
{'cosqa': {'NDCG': {'NDCG@1': 0.318, 'NDCG@3': 0.43193, 'NDCG@5': 0.47428, 'NDCG@10': 0.52433, 'NDCG@100': 0.56868, 'NDCG@1000': 0.57202}, 'MAP': {'MAP@1': 0.318, 'MAP@3': 0.403, 'MAP@5': 0.4267, 'MAP@10': 0.44752, 'MAP@100': 0.45714, 'MAP@1000': 0.45727}, 'Recall': {'Recall@1': 0.318, 'Recall@3': 0.516, 'Recall@5': 0.618, 'Recall@10': 0.772, 'Recall@100': 0.974, 'Recall@1000': 1.0}, 'Precision': {'P@1': 0.318, 'P@3': 0.172, 'P@5': 0.1236, 'P@10': 0.0772, 'P@100': 0.00974, 'P@1000': 0.001}}}
Run failed: cannot unpack non-iterable NoneType object
Run failed: cannot unpack non-iterable NoneType object
Run failed: cannot unpack non-iterable NoneType object
Run failed: cannot unpack non-iterable NoneType object


In [ ]:
duplicate_code_groups = df.groupby('code').filter(lambda x: len(x) > 1)
num_duplicate_code_groups = duplicate_code_groups['code'].nunique()
print(f"Number of code groups with >1 item: {num_duplicate_code_groups} out of {len(df)}")

In [ ]:
duplicate_code_df = df.groupby('code').filter(lambda x: len(x) > 1)

result = (
    duplicate_code_df
    .groupby('code')
    .agg({
        'corpus_id': lambda x: list(set(x)),
        'query_id': lambda x: list(set(x))
    })
    .reset_index()
    .rename(columns={'corpus_id': 'corpus_ids_with_same_code', 'query_id': 'query_ids_for_those_corpus'})
)

print(result.head())


                                                code  \
0  async def _send_plain_text(self, request: Requ...   
1  async def _thread_coro(self, *args):\n        ...   
2  async def async_input(prompt):\n    """\n    P...   
3  async def async_run(self) -> None:\n        ""...   
4  async def cursor(self) -> Cursor:\n        """...   

                           corpus_ids_with_same_code  \
0              [d3961, d1874, d20448, d6227, d10337]   
1  [d6144, d118, d5150, d12781, d7622, d1889, d82...   
2  [d2781, d16258, d16413, d4944, d9133, d9137, d...   
3           [d19465, d20020, d16870, d19314, d19313]   
4                   [d16659, d18387, d19386, d17979]   

                          query_ids_for_those_corpus  
0              [q20448, q1874, q3961, q6227, q10337]  
1  [q11702, q5150, q6147, q7622, q8299, q118, q18...  
2  [q4944, q16413, q9137, q16258, q20250, q9133, ...  
3           [q20020, q19314, q16870, q19465, q19313]  
4                   [q17979, q19386, q16659, q18387

In [ ]:
duplicate_code_groups = df.groupby('cleaned_code').filter(lambda x: len(x) > 1)

num_duplicate_code_groups = duplicate_code_groups['cleaned_code'].nunique()

print(f"Number of cleaned_code groups with >1 item: {num_duplicate_code_groups} out of {len(df)}")

Number of cleaned_code groups with >1 item: 3406 out of 20604


In [ ]:
duplicate_code_groups = df.groupby('explanation_deepseek_1_cleaned').filter(lambda x: len(x) > 1)

num_duplicate_code_groups = duplicate_code_groups['explanation_deepseek_1_cleaned'].nunique()

print(f"Number of explanation_deepseek_1_cleaned groups with >1 item: {num_duplicate_code_groups} out of {len(df)}")

Number of explanation_deepseek_1_cleaned groups with >1 item: 93 out of 20604
